In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests 
import xml 
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time
import re



In [2]:
driver =webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://steamdb.info/sales/')

time.sleep(2)  # Allow 2 seconds for the web page to open
scroll_pause_time = 1 # You can set your own pause time. My laptop is a bit slow so I use 1 sec
screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
i = 1

select = Select(driver.find_element_by_name('DataTables_Table_0_length'))

# select by visible text
select.select_by_visible_text('1K')

while True:
    # scroll one screen height each time
    driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
    i += 1
    time.sleep(scroll_pause_time)
    # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
    scroll_height = driver.execute_script("return document.body.scrollHeight;")  
    # Break the loop when the height we need to scroll to is larger than the total scroll height
    if (screen_height) * i > scroll_height:
        break 



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [/Users/workplace/.wdm/drivers/chromedriver/mac64/96.0.4664.45/chromedriver] found in cache


In [3]:
data = driver.find_elements_by_class_name('app')



In [4]:
cols=['name','price','rating','dis_ends','dis_start','release','discount']
df=pd.DataFrame(columns=cols)
df

,name,price,rating,dis_ends,dis_start,release,discount


In [6]:
for d in data:
    td=d.find_elements_by_css_selector('td')
    name=td[2].text
    discount=td[3].text
    price=td[4].text
    rating=td[5].text
    dis_ends=td[6].text
    dis_start=td[7].text
    release=td[8].text
    
    row_dict = {'name':name,'price':price,'rating':rating,'dis_ends':dis_ends,'dis_start':dis_start,'release':release,'discount':discount} 
    df = df.append(row_dict, ignore_index = True)

In [7]:
df.shape

(987, 7)

In [8]:
df.head(20)

,name,price,rating,dis_ends,dis_start,release,discount
0,Little Inferno\nnew historical low,2.99 SR,91.28%,in 3 days,4 hours ago,9 years ago,-90%
1,Human Resource Machine\nnew historical low,7.48 SR,90.11%,in 3 days,4 hours ago,6 years ago,-75%
2,HIVESWAP: ACT 1\nnew historical low,6.28 SR,89.56%,in 2 days,4 hours ago,4 years ago,-65%
3,7 Billion Humans\nnew historical low,7.48 SR,88.34%,in 3 days,4 hours ago,3 years ago,-75%
4,Pesterquest\nnew historical low,8.48 SR,86.06%,in 2 days,4 hours ago,2 years ago,-66%
5,HIVESWAP: ACT 2\nnew historical low,22.46 SR,82.57%,,,,-25%
6,The Life and Suffering of Sir Brante,22.71 SR,88.73%,,,,-35%
7,Hiveswap Friendsim\nlowest price is 1.37 SR at...,9.16 SR,88.26%,,,,-66%
8,Accident,17.97 SR,76.57%,,,,-40%
9,Teardown,31.96 SR,93.26%,in 3 days,yesterday,13 months ago,-20%


In [9]:
df.to_csv('steam_discounts.csv')